## **Student's Autograder**
This is a project to calculate quiz and term scores from q&a pdf and response pdf.

Whats left: 

* Editing pdf

### Importing the required libraries 

In [5]:
import os
import pandas as pd
import numpy as np
import tabula
from ipywidgets import widgets
from PyPDF2 import PdfFileReader, PdfFileWriter
from PyPDF2.generic import DecodedStreamObject, EncodedStreamObject,ArrayObject

In [6]:
#debugging for notebooks current directory for reading files later.
import os
os.getcwd()

'C:\\Users\\506761\\OneDrive - Tata Steel Limited\\Documents\\Python Scripts'

### Reading pdfs

In [7]:
resp_pdf = widgets.Text(
 placeholder='Enter filename with extension',
 description='Response pdf:'
 )
qa_pdf = widgets.Text(
 placeholder='Enter filename with extension',
 description='Q&A pdf:'
 )
# POD22S1C31570014.pdf
# POD22S1C31590017.pdf
# POD22S1C11550029.pdf
# POD22S1C11320038.pdf
# IIT M DIPLOMA QUIZ1 EXAM QPF1.pdf
# IIT M DIPLOMA QUIZ2 EXAM QPF1.pdf
# IIT M QUIZ 1 FOUNDATION DAD DIPLOMA QPD1.pdf
display(resp_pdf,qa_pdf)

Text(value='', description='Response pdf:', placeholder='Enter filename with extension')

Text(value='', description='Q&A pdf:', placeholder='Enter filename with extension')

In [8]:
df = tabula.read_pdf(resp_pdf.value,pages='all',multiple_tables=False)
filename=qa_pdf.value

In [9]:
df[0].tail()

,Name,SHEKHAR MOHANTY HEMANT
255,640653351773,Unanswered
256,640653351768,Unanswered
257,640653351769,Unanswered
258,640653351775,Unanswered
259,640653351776,Unanswered


### Variables initialization

In [10]:
my_question="Question ID"
my_answer="Options Selected"
unanswered="Unanswered"
file_qid="Question Id"

In [11]:
df[0].iloc[4].values

array(['Question ID', 'Options Selected'], dtype=object)

In [12]:
df[0].head()

,Name,SHEKHAR MOHANTY HEMANT
0,QP Set,2022 July: IIT M DIPLOMA QUIZ2 EXAM QPF1
1,Test Center Name,iON Digital Zone iDZ Jamshedpur
2,Shift,AN
3,Roll No,POD22S1C31590017
4,Question ID,Options Selected


In [13]:
df1=df[0].replace(unanswered,np.nan,regex=True)

Students response for exam dataframe

In [14]:
myans=df1.dropna()
myans.columns=myans.iloc[4].values
myans=myans[5:]

In [15]:
myans[my_question]=myans[my_question].apply(int)
myans=myans.sort_values(my_question)
print(len(myans))
myans.tail()

42


,Question ID,Options Selected
235,640653351751,"6406531167331,6406531167334"
234,640653351752,6406531167338
239,640653351754,80
240,640653351755,90
241,640653351756,10


### Useful functions

In [16]:
def gentextarr(content):
    """
    Returns an array of text block properties of pdf
    Parameter:
        content: string object of page content obtained by page.getContents().getData().decode() 
    """
    lines = content.splitlines()
    result = []
    in_text = False
    for line in lines:
        if line == "BT":
            in_text = True
            temp=[]

        elif line == "ET":
            in_text = False
            result.append(temp)
            temp=[]

        elif in_text:
            if line[-2:].lower()=="tj" or line[-2:].lower()=="tf" or line[-2:].lower()=="rg":
                temp.append(line[:])
    return result

def gen_cmap(font,pg):
    """
    Returns a character map dictionary of a pdf 'filename' page 'pg' and font 
    Parameter:
     font: font key name in meta properties of pdf.
     pg: int page number of pdf
    """
    # pdf = PdfFileReader(filename)
    # page = pdf.getPage(pg)
    page=pg
    temp=False
    cmap={}
    for i in (page.getObject()['/Resources']['/Font'][font]['/ToUnicode'].getObject().getData().decode().split('\n')):
        if "endbfrange" in i:
            temp=False
        if temp:
            cmap[i[1:5]]=i[-5:-1]
        if "beginbfrange" in i:
            temp=True
    return cmap

def font_decoder(text,cmap):
    """
    Returns decoded word from pdf text hex code. 
    """

    count=0
    word=""
    temp=""
    for i in text:
        count+=1
        if count==4:
            temp+=i
            word+=chr(int(cmap.get(temp.lower(),temp.lower()),16))
            temp=""
            count=0
        else:
            temp+=i
    return word


## PDF Content

In [17]:
"""
This piece of code extracts question answer pdf text content as well as correct answer key.
"""
pdf = PdfFileReader(filename)
sum=0
df=[[file_qid,"Correct Marks","Answers"]]
m_bool=False
marks=""
ans_bool=False
ans=""
row=[]
temp=[]
s_head=[]
temp_head=""
ishead=False
for p in range(0,pdf.getNumPages()):
    content=pdf.pages[p].getContents()
    #a different type content
    if type(content)==ArrayObject:
        m=""
        for n in content:
            m+=n.getObject().getData().decode()
        content=m
    elif type(content)==EncodedStreamObject or type(content)==DecodedStreamObject:
        content=content.getData().decode()
    for i in gentextarr(content):
        text=""
        ans_bool=False
        for j in i:
            # getting character map for a font type
            if j[-2:].lower()=="tf":
                cmap=gen_cmap(j.split()[0],pdf.pages[p])
                #checking for subject headings with font size =18
                if j.split()[1]=="18":
                    ishead=True
            # checking green color as ind of correctness        
            if j=="0 0.50196 0 rg" and len(row)>1:
                ans_bool=True
    
            elif j[-2:].lower()=="tj":
                text=font_decoder(j[1:-3],cmap)
                if ishead:
                    ishead=False
                    temp_head=text
                
                if (not m_bool) and (not ans_bool):
                    # checking for question id
                    if str(file_qid) in text:
                        if len(temp)>0:
                            row.append(temp)
                            df.append(row)
                        row=[]
                        temp=[]
                        if text.split()[7].isnumeric():
                            s_head.append(temp_head)
                            row=[text.split()[7]]
                            m_bool=True
                    
                # extracting marks
                elif m_bool:
                    if "Correct Marks" in text:
                        # appending to heading list if marks is 0
                        # if text.split()[-1]=="0":
                        #     s_head.append(temp_head)
                        if len(row)==1:
                            row.append(text.split()[-1])
                        m_bool=False
                        
                #extracting answer key
                elif ans_bool:
                    if text!=" ":
                        if len(text)>=13 and text[:13].isnumeric():
                            temp.append(text[:13])
                        elif len(text.split())==3 and "to" in text.split()[1]:
                            t=text.split()
                            temp=[float(t[0]),float(t[-1])]
                        elif len(temp)==0:
                            temp.append(text)
if len(temp)>=1:
    row.append(temp)
    df.append(row)

In [18]:
print(len(df))
df[1]

261


['640653351495', '0', ['6406531166580']]

In [19]:
df2=pd.DataFrame(df[1:],columns=df[0])
df2[file_qid]=df2[file_qid].apply(int)
df2["Subjects"]=np.array(s_head)
df2=df2.sort_values([file_qid])
df2.head()

,Question Id,Correct Marks,Answers,Subjects
0,640653351495,0,[6406531166580],Maths 2
1,640653351496,2,"[6406531166582, 6406531166583]",Maths 2
6,640653351498,1,[0],Maths 2
7,640653351499,1,[-4],Maths 2
8,640653351500,1,[-2],Maths 2


In [20]:
def evaluate(df2):
    
    #Multi option
    df2['Score'] = df2.apply(lambda x: (len(set(x["My answer"].split(","))\
                                            .intersection(set(x["Answers"])))/len(x["Answers"]))*float(x['Correct Marks'])\
                             if type(x["Answers"][0])==str and len(x["Answers"][0])==13 and\
                                len(set(x["My answer"].split(",")))<=len(set(x["Answers"])) and\
                                set(x["My answer"].split(",")).issubset(set(x["Answers"]))\
                             else x["Score"], axis=1)
    #integer ranged
    df2['Score'] = df2.apply(lambda x: float(x['Correct Marks'])\
                             if len(x["My answer"])<13 and len(x["My answer"])>0 and\
                                len(x['Answers'])>=1 and\
                                float(x['My answer']) >= float(x["Answers"][0]) and\
                                float(x['My answer']) <= float(x["Answers"][-1])\
                             else x["Score"], axis=1)
    return df2

## Final score

### Method 1: using index of 0 correct marks

In [21]:
# s_index=df3[df3["Correct Marks"]=="0"].index
# s_index

In [22]:
# subjects={}
# for i in range(len(s_index)):
#     if i==len(s_index)-1:
#         subjects[df3["Subjects"][s_index[i]]]=df3.iloc[s_index[i]:]
#     else:
#         subjects[df3["Subjects"][s_index[i]]]=df3.iloc[s_index[i]:s_index[i+1]]
# subjects.keys()

In [23]:
# reports=[]
# for i in subjects.values():
#     print(i["Correct Marks"].apply(float).values)
#     reports.append([i["Subjects"].unique()[0],i["Correct Marks"].apply(float).values.sum(),i["Score"].sum()])

In [24]:
# for i in reports:
#     print(i[0],":",i[2]/i[1]*100,"%")

### Method 2: Using groupby function

In [25]:
df3=df2.set_index('Question Id').combine_first(myans.set_index('Question ID'))\
       .reset_index().replace(np.NAN,"")
df3.columns=["Question Id","Answers","Correct Marks","My answer","Subjects" ]
df3["Score"]=np.zeros(df3.shape[0])
df3=evaluate(df3)


In [26]:
df3[df3["Score"]>0]

,Question Id,Answers,Correct Marks,My answer,Subjects,Score
129,640653351638,[6406531166957],3,6406531166957,Java,3.0
130,640653351639,[6406531166963],3,6406531166963,Java,3.0
131,640653351640,"[6406531166964, 6406531166966]",4,"6406531166964,6406531166966",Java,4.0
132,640653351641,[6406531166970],3,6406531166970,Java,3.0
133,640653351642,"[6406531166972, 6406531166973]",4,"6406531166972,6406531166973",Java,4.0
135,640653351644,[6406531166983],3,6406531166983,Java,3.0
137,640653351646,[6406531166989],3,6406531166989,Java,3.0
138,640653351647,[6406531166994],3,6406531166994,Java,3.0
139,640653351648,[6406531166997],3,6406531166997,Java,3.0
141,640653351650,"[6406531167005, 6406531167007]",4,"6406531167005,6406531167007",Java,4.0


In [27]:
df3["Correct Marks"]=df3["Correct Marks"].apply(float)

In [28]:
report2=pd.DataFrame(df3.groupby("Subjects")[["Correct Marks","Score"]].sum())

In [29]:
report2["Percentage"]=(report2["Score"]*100/report2["Correct Marks"]).round(2)

In [30]:
report2.columns=["Total Marks","Scored","Percentage"]
headers = {
    'selector': 'thead',
    'props': 'background-color: #0dcaf0; color: white;'
}
report2.style.applymap(lambda v: 'color:red;' if (v == 0) else 'color:green',subset=["Percentage","Scored"])\
    .set_table_styles([headers])\
    .format({"Total Marks":'{:.2f}',"Scored":'{:.2f}',"Percentage":'{0:,.2f}%'})

,Total Marks,Scored,Percentage
Subjects,,,
AppDev-1,50.00,0.00,0.00%
AppDev-2,50.00,24.50,49.00%
BDM,20.00,0.00,0.00%
Business Analytics,20.00,11.00,55.00%
CT,50.00,0.00,0.00%
DBMS,50.00,0.00,0.00%
Java,50.00,39.00,78.00%
MLF,50.00,0.00,0.00%
MLP,50.00,0.00,0.00%


In [31]:
report2[report2["Percentage"]>0]

,Total Marks,Scored,Percentage
Subjects,,,
AppDev-2,50.0,24.5,49.0
Business Analytics,20.0,11.0,55.0
Java,50.0,39.0,78.0


### Exporting dataframe as csv

In [32]:
df3[df3["Subjects"].isin(report2[report2["Percentage"]>0].index)].to_csv("summary.csv")
report2[report2["Percentage"]>0].to_csv("report_card.csv")